<a href="https://colab.research.google.com/github/freddyacuna/webscraping-contacto-empresas-chilenas/blob/main/web_scraping_empresas_contacto_Chile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción


Objetivo: este proyecto consiste en extraer información de contacto (nombre, telefono, mail de contacto) de empresas que se encuentran registradas en SII. Se utiliza python para su ejecución.

El código automatiza la búsqueda masiva de la información. Los pasos que deberá tener la busqueda son:



1.   Abrir una pestaña del buscador
2.   Ir a la pagina de google maps
3.   Ingresar el nombre de la empresas que aparece en nuestra bbdd del SII (filtrada para empresas excluye instituciones publicas y personas naturales).
4.   Abrir la coincidencia del buscador de google maps.
5.   Extraer los datos (nonmbre, telefono, sitio web) y guardarlos en un googlesheet.
6.   Si existe sitio web y se logra una coincidencia entre la busqueda de maps y el nombre de la empresa, procedemos a re-direccionar al sitio web.
7.   Realizar escrapeo del sitio web para extraer el mail contacto de al menos 15 pestañas del sitio. Luego se guarda en googlesheet los mail de contactos 
8.   Repetir el procedimiento N donde N corresponde al total de empresas registradas en el SII luego de filtrada la bbdd.


Por otro lado, será interesante replicar este mismo procedimiento usando R. Los paquetes relevantes para su aplicación son:



*   Realizar web scraping mediante el paquete [rvest] y [polite].
*   Acceder y modificar información en planillas de Google Sheets mediante el [googlesheets4].
*   Limpiar y modificar caracteres mediante expresiones regulares ([stringr])


Si le interesa seguir este camino, no dude en contactarse con: 
1. [Github Freddy Acuña](https://github.com/freddyacuna)
2. [Linkedin Freddy Acuña](https://www.linkedin.com/in/freddyacuna/)




Conectamos con google colab y drive para acceder a los archivos de mi nube

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Conectando con googlesheet (**learning**)

Almacenaremos la información de scrapeo en un googlesheet. Una forma inicial de como trabajar con hojas de cálculo de google es la siguiente:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
###############################################
from google.colab import auth
auth.authenticate_user()
###############################################
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
wb = gc.open("AB_NYC_2019")

In [ ]:
sheet = wb.worksheet('AB_NYC_2019')

## Re-conectar (**learning**)

Existe la posibilidad de no desconectar el ambiente para que el codigo no se detenga. Con el siguiente métoodo se podrá dar solución [Re-conectar](https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting)

Deberá ir a : Inspeccionar> Consola > Ingresar al menos uno de los dos codigos que siguen.

In [ ]:
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)

In [ ]:
function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("Put ID here").click() // Change id here
}
setInterval(ClickConnect,60000)

# Web Scraping usando : Selenium + Beautiful Soup 

Utilizamos dos librerias para la extración de web scraping. 

1) [Selenium - 1](https://selenium-python.readthedocs.io/) [Selenium 2](https://pypi.org/project/selenium/): Se usa para navegar por los sitios. Abrir pestaña, seleccionar los codigos relevantes en el sitio de google maps para acceder a los items relevantes (nombre, telefono, mail, sitio web)

2) [beautiful soup](https://pypi.org/project/beautifulsoup4/): Se usa para navegar en el sitio web re-direccionado para escrapear los mail de contactos.


Algunas paginas de referencia que guiaron el camino usando estas librerias en python:


https://medium.com/@rodrigonader/web-scraping-to-extract-contact-information-part-1-mailing-lists-854e8a8844d2

https://medium.com/nerd-for-tech/linked-in-web-scraper-using-selenium-15189959b3ba

https://gist.github.com/lobstrio/b95408a2ed7d01a64e704c417aab64f3

https://www.linkedin.com/pulse/how-easy-scraping-data-from-linkedin-profiles-david-craven/




## Librerias

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Actual/INNOVA CHILE - CORFO/BBDD/Portafolio/Web Scraping/data')
#os.getcwd()

In [ ]:
!pip install selenium
!pip install pyxlsb
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

from google.colab import auth
auth.authenticate_user()

import gspread ## PERMITE CONECTAR CON EL API DE GOOGLE
from oauth2client.client import GoogleCredentials

import pyxlsb
import time # Lo utilizaremos para que no detecte que somos un bot
import random

from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import re                           #expresiones regulares
import requests                     #para enviar solicitudes HTTP
from urllib.parse import urlsplit   #dividir las URL en partes componentes
from collections import deque       #contenedor similar a una lista con anexos 
from bs4 import BeautifulSoup       #extraer datos de archivos HTML de sitios web
import pandas as pd

     |████████████████████████████████| 904 kB 6.8 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates 

## Code

In [ ]:
class ScrapearGMaps:
    
    data = {} # Info que extreré de las empresas
    worksheet = {}
    
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome('chromedriver',options=options)
        
        now = datetime.now()
        today = now.strftime("%Y-%m-%d")

        gc = gspread.authorize(GoogleCredentials.get_application_default())      # Conectamos con nuestra cuenta de google drive

        # Abrir por titulo
        sh = gc.open("Empresas Contacto")
        #sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Gvdp_0QDu5wy683i0vx9dfeQo1GoTX-eC3uPLbAEDr8/edit?usp=sharing')
  

        # SSeleccionar primera hoja para ir añandiendo las cosas
        self.worksheet = sh.get_worksheet(0)
        #self.worksheet = sh.worksheet('bbdd')
    

    # Extraera Nombre de la empresa    
    def get_name(self):
        try:
            return self.driver.find_element_by_xpath("//h1[contains(@class,'header-title')]").text
        except:
            return ""


    # Extraera telefono de la empresa     
    def get_phone(self):
        try:
            return self.driver.find_element_by_xpath("//button[contains(@data-item-id,'phone')]").text
        except:
            return ""

    # Extraera sitio web de la empresa     
    def get_website(self):
        try:
            return self.driver.find_element_by_css_selector("[data-item-id='authority']").text
        except:
            return ""

##      M     A     I     L

    # Extraera mail de la empresa    
    def get_email(self, url_website):
        try:
            starting_url = url_website
            unprocessed_urls = deque([starting_url])
            processed_urls = set()
            emails = set()

            while len(unprocessed_urls):
                if len(processed_urls)>15:      # limite para evitar las redirecciones en sitios web: e-commerce + otras páginas
                      break
                url = unprocessed_urls.popleft()
                processed_urls.add(url)
                parts = urlsplit(url)
                base_url = "{0.scheme}://{0.netloc}".format(parts)
                path = url[:url.rfind('/')+1] if '/' in parts.path else url

                #print("Crawling URL %s" % url)
                try:
                    response = requests.get(url)
                except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
                    continue
                
                new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[cl|com]+", response.text, re.I)) #[. In | .com | .uk] @ + \ S +
                emails.update(new_emails)
                #print(emails)

                soup = BeautifulSoup(response.text, 'lxml')

                for anchor in soup.find_all("a"):
                    link = anchor.attrs["href"] if "href" in anchor.attrs else ''
                    if link.startswith('/'):
                        link = base_url + link
                    elif not link.startswith('http'):
                        link = path + link
                    if not link in unprocessed_urls and not link in processed_urls:
                        unprocessed_urls.append(link)
                    



            return ','.join(emails)   # (contacto@chile.cl, contacto2 , )
            
        except:
            return ""
    
    def scrape(self, url):
        try:
            self.driver.get(url)        ## Accedemos a la URL
            #print(self.driver.page_source)
            time.sleep(1)
            
            #element = self.driver.find_element_by_xpath("//button[.//span[text()='Acepto']]")        ## solucion al problemas de cookies si estamos trabajando en un entorno local. Es una ruta relativa y no absoluta
            #element.click()
            
            time.sleep(1)
                        
      
            name = self.get_name()
            address = self.get_address()
            phone_number = self.get_phone()
            website = self.get_website()
            coords = self.get_geocoder(self.driver.current_url)
            email = ""
            if website != "":
                email = self.get_email('http://'+website)
                
                
            #print([name, address, phone_number, coords[0], coords[1], website, email])
                
                
            row_index = len(self.worksheet.col_values(1)) + 1
            self.worksheet.update_acell('A'+str(row_index), name)
            self.worksheet.update_acell('B'+str(row_index), '#'+phone_number)
            self.worksheet.update_acell('C'+str(row_index), website)
            self.worksheet.update_acell('D'+str(row_index), email)
            self.worksheet.update_acell('E'+str(row_index), query)                  
                
 
            time.sleep(1)
            
        except Exception as e:
            print(e)
        
        time.sleep(3)
        #self.driver.quit()

        return(self.data)

## Consulta

### BBDD empresas completas

Cargamos la bbdd empresas sii campo "tipo de contribuyente".

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Actual/INNOVA CHILE - CORFO/BBDD/Portafolio/Web Scraping/data/PUB_Empresas_2019_102020.xlsb', engine='pyxlsb')

In [ ]:
df.shape

(673847, 20)

### BBDD empresas filtrado

Se limpia por termino de giros y solo se mantienen empresas juridicas

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Actual/INNOVA CHILE - CORFO/BBDD/Portafolio/Web Scraping/data/PUB_Empresas_2019_102020_personas_juridicas.xlsb', engine='pyxlsb')

In [ ]:
df.shape

(636696, 20)

### Busqueda Razón Social (**learning**)

In [ ]:
name = df['Razón social'][0]
name

'CENTRO DE MATERIALES DE CONSTRUCCION FERNANDEZ S A'

In [ ]:
query = name
url = "https://www.google.cl/maps?q="+query.replace(" ", "+")+"&hl=es"
print(url)

https://www.google.cl/maps?q=CENTRO+DE+MATERIALES+DE+CONSTRUCCION+FERNANDEZ+S+A&hl=es


Obs: 

1) empresas no cuentan con sitio web.


2) si exiten más enlaces (+1) sol:

3) empresas no existen en la busqueda de google maps

In [ ]:
for i in range(0,3):
  print(i)

0
1
2


### Tramos

In [ ]:
 for i in range(0,636696):
   query = df['Razón social'][i]
   url = "https://www.google.cl/maps?q="+query.replace(" ", "+")+"&hl=es"
   gmaps = ScrapearGMaps()
   gmaps.scrape(url)